# Rede Neural Convolucional (Convolutional Neural Network - CNN)
## Autor: Mariano F.M.A.S.
### Data: 09/07/2023


Neste notebook, exploramos como treinar e avaliar um modelo de Rede Neural Convolucional (CNN) usando PyTorch. Reaproveitamos as função construidas em `1_neural_network_MNIST_digit_recognizer_MLP` para treinar e calcular as métricas de desempenho.


# Rede Neural Convolucional (Convolutional Neural Network - CNN)

Uma Rede Neural Convolucional (Convolutional Neural Network - CNN) é um tipo de rede neural artificial projetada para processar dados com uma estrutura de grade semelhante, como uma imagem. As CNNs são uma das principais categorias de modelos para processamento de imagens e visão computacional.

As CNNs são compostas por uma ou mais camadas convolucionais, seguidas por uma ou mais camadas totalmente conectadas (como em uma rede neural multicamada). As camadas convolucionais criam mapas de características que registram uma região da imagem, enquanto as camadas totalmente conectadas minimizam o erro na classificação.

Aqui estão os principais componentes de uma CNN:

1. **Camada Convolucional:** Esta camada usa um conjunto de filtros e uma operação de convolução para criar um mapa de características.

2. **Função de Ativação / Não-linearidade (ReLU, Sigmoid, etc.):** Após cada camada convolucional, a CNN aplica uma função de ativação não linear. A função de ativação mais comum é a ReLU (Rectified Linear Unit).

3. **Camada de Pooling ou Subamostragem:** Após a função de ativação, a CNN aplica uma operação de pooling para reduzir a dimensionalidade do mapa de características e evitar o overfitting. A operação de pooling mais comum é a max pooling.

4. **Camada Totalmente Conectada (FC):** Após várias camadas convolucionais e de pooling, a CNN usa uma ou mais camadas totalmente conectadas para classificação.

5. **Função de Perda (Softmax, Cross Entropy, etc.):** A última camada de uma CNN é geralmente uma camada softmax ou uma camada de entropia cruzada que é usada para a classificação de saída.

As CNNs têm sido muito bem-sucedidas em tarefas de visão computacional, como classificação de imagens, detecção de objetos, reconhecimento facial e muito mais.


<img src="https://slideplayer.com.br/slide/15363722/93/images/10/Convolutional+Neural+Network+%28CNN%29.jpg" alt="CNN." width="700"  >
 




In [ ]:
import os
import torch
import numpy as np
import scipy
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import label_binarize
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, matthews_corrcoef, cohen_kappa_score, classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc  
#from itertools import cycle 
 
    
# Checking GPU is available 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Training on device: ", device)

random_seed = 11

# Carregando os dados

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Importando os dados

# Dados de treinamento 
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
#Separa os entre train e test
train, val = train_test_split(data, test_size=0.2)

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

# Dados para validação do modelo
test_data  = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')



## Criando os dataloaders da CNN

Agora estaremos usando os dados como imagens (matrizes de pixels) invés de vetores (1D), então modificamos o retorno da função Dataset.

O método `reshape()` é usado para alterar a forma de um array sem alterar seus dados. Ele retorna um array que contém os mesmos dados que o array original, mas com uma nova forma.

No seu caso, `.reshape((1, 28, 28))` está alterando a forma do array para ter uma forma de `(1, 28, 28)`. Isso significa que o array resultante terá três dimensões, com o tamanho da primeira dimensão sendo 1, o tamanho da segunda dimensão sendo 28 e o tamanho da terceira dimensão sendo 28.

Em termos de imagens, isso geralmente é usado para formatar uma imagem plana 1D em uma imagem 2D. No conjunto de dados MNIST cada imagem é de 28x28 pixels, mas as imagens são armazenadas como arrays 1D de 784 elementos. Então, `.reshape((1, 28, 28))` está reformatando essa imagem 1D em uma imagem 2D de 28x28 pixels. A dimensão extra (1) é usada para indicar o número de canais de cor na imagem. Neste caso, é 1 porque as imagens MNIST são em escala de cinza e, portanto, têm apenas um canal de cor.



In [ ]:
# Definindo a classe do dataset
class MNISTDataset_CNN(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        #Usando o reshape
        image = torch.tensor(self.data.iloc[index, 1:].values.astype(float).reshape((1, 28, 28))/255.0, dtype=torch.float32) 
        label = torch.tensor(self.data.iloc[index, 0], dtype=torch.long)
        
        return image, label

# Criando os dataloaders
train_dataset_CNN = MNISTDataset_CNN(train)
val_dataset_CNN = MNISTDataset_CNN(val)

train_loader_CNN = DataLoader(train_dataset_CNN, batch_size=64, shuffle=True)
val_loader_CNN = DataLoader(val_dataset_CNN, batch_size=64, shuffle=True)



Se `shuffle=True`, os dados serão embaralhados antes de cada época. Isso é útil para garantir que o modelo não aprenda nada das sequências específicas dos dados de entrada. Em outras palavras, embaralhar os dados garante que o modelo não seja influenciado pela ordem dos exemplos de treinamento.

Se `shuffle=False`, os dados serão passados na mesma ordem a cada época. Isso pode ser útil em certos casos, como quando você está trabalhando com séries temporais e a ordem dos dados é importante.

Em geral, é uma boa prática embaralhar os dados de treinamento para garantir que o modelo seja robusto e não aprenda a fazer previsões com base na ordem dos exemplos.

In [ ]:
# Definindo a classe do dataset
class MNISTDataset_test_CNN(Dataset):
    def __init__(self, test_data):
        self.test_data = test_data
    
    def __len__(self):
        return len(self.test_data)
    
    def __getitem__(self, index): 
        image = torch.tensor(self.test_data.iloc[index, :].values.astype(float).reshape((1, 28, 28))/255.0, dtype=torch.float32) 
       
        return image

# Criando os dataloaders
test_dataset_CNN = MNISTDataset_test_CNN(test_data)
 
test_loader_CNN = DataLoader(test_dataset_CNN, batch_size=64, shuffle=False)


## Visualizando os Dados de Treinamento e Test 

In [ ]:
fig, axis = plt.subplots(3, 5, figsize=(10, 7))

for images, labels in train_loader_CNN:
    for i, ax in enumerate(axis.flat):
        image, label = images[i], labels[i]
        
        ax.imshow(image.view(28, 28), cmap='binary') # add imagem
        ax.set(title = f"{label}") # add label
    break  


# Função de Treinando

Definimos uma função para fazer o treinamento de um modelo qualquer.

Essa função vai treinar o modelo por infinitas epochs, até que a acurácia pare de aumentar após um determinado número de epocas dada por `patience`.

Após cada epoch, avaliamos o modelo usando o conjunto de validação.


In [ ]:
 
def train_model(model, criterion, optimizer, train_loader, val_loader, patience):
    best_accuracy = 0.0  # Inicialize a melhor acurácia como 0
    epochs_without_improvement = 0  # Contador para épocas sem melhoria
    #patience => Número de épocas para esperar, depois de obter a melhor acurácia, antes de parar
    model.train()
    epoch= 0
    
    while True:                      
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)   
             
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()  
            
        # Validação
        accuracy, validation_loss = validate(model, val_loader, criterion)
        print('Epoch [{}], Loss: {:.4f}, Validation Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, loss.item(), validation_loss, accuracy))
        epoch=epoch+1
        
        # Verifica se a acurácia melhorou
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            epochs_without_improvement = 0  # Reseta o contador
            torch.save(model.state_dict(), 'best_model_CNN.pth') # Salva o modelo com a melhor acurácia
        else:
            epochs_without_improvement += 1 # Incrementar o contador
         # Se a acurácia não melhorar por um número 'patience' de épocas, para de treinar
        if epochs_without_improvement == patience:
            print('Stopping training!')
            break
 

In [ ]:
def validate(model, val_loader, criterion):
    model.eval()  # Defina o modelo para o modo de avaliação
    correct = 0
    total = 0
    validation_loss = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)   #Encontra o índice da probabilidade máxima
            # A função torch.max() retorna dois valores: o primeiro é o valor máximo encontrado em cada coluna do tensor
            #e o segundo é o índice do valor máximo em cada coluna. ex.: [0.2, 0.4, 0.9, 0.5] => (0.9, 2)= (valor máximo,índice)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = (correct / total) * 100
    validation_loss = validation_loss / len(val_loader)  # Media de loss

    model.train()  # Defin3 o modelo de volta ao modo de treinamento

    return accuracy, validation_loss



# Construindo um Modelo CNN

No PyTorch, existem duas maneiras principais de definir uma Rede Neural Convolucional (CNN):

1. **Usando `nn.Sequential`**: Esta é uma maneira mais simples e mais direta de definir uma CNN. Você pode definir todas as camadas da rede em uma única linha de código. Aqui está um exemplo:
 
2. **Usando `nn.Module`**: Esta é uma maneira mais flexível de definir uma CNN, pois permite um controle mais granular sobre o fluxo de dados através da rede. Aqui está um exemplo:

 
Ambas as abordagens resultarão em uma CNN funcional, mas a escolha entre elas depende das necessidades específicas do seu projeto. Se você precisar de um controle mais detalhado sobre a passagem para a frente da sua rede (por exemplo, se você quiser usar conexões residuais ou saltos), então a subclassificação `nn.Module` será mais apropriada. Se a sua rede for relativamente simples e direta, então `nn.Sequential` pode ser uma opção mais conveniente.
## Modelo CNN_1 - nn.Sequential

In [ ]:
class MNISTModel_CNN_1(nn.Module):
    def __init__(self):
        super(MNISTModel_CNN_1, self).__init__()

        # Camadas convolucionais
        self.conv_layers = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ])

        # Calcula a dimensão de entrada das camadas totalmente conectadas
        self.fc_input_dim = self.calculate_fc_input_dim()

        # Camadas totalmente conectadas
        self.fc_layers = nn.ModuleList([
            nn.Linear(self.fc_input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(), 
            nn.Linear(128, 10)
        ])

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)

        x = x.view(-1, self.fc_input_dim)
        for layer in self.fc_layers:
            x = layer(x)

        return x

    def calculate_fc_input_dim(self):
        # Cria uma entrada de exemplo e passa pelas camadas convolucionais
        example_input = torch.zeros(1, 1, 28, 28)
        for layer in self.conv_layers:
            example_input = layer(example_input)
            #output_shape = example_input.shape
            #print(output_shape)
        # Obtém a forma da saída das camadas convolucionais
        output_shape = example_input.shape
        fc_input_dim = output_shape[1] * output_shape[2] * output_shape[3]
        return fc_input_dim
    
model_CNN_1=MNISTModel_CNN_1()
print(model_CNN_1)

 

## Modelo CNN_2 - nn.Module

In [ ]:
'''

class MNISTModel_CNN_2(nn.Module):
    def __init__(self):
        super(MNISTModel_CNN_2, self).__init__()

        # Primeira camada convolucional
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Segunda camada convolucional
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)


        # Camadas totalmente conectada
        self.fc_input_dim = self.calculate_fc_input_dim() # Calcula a dimensão de entrada das camadas totalmente conectadas
        self.fc1 = nn.Linear(self.fc_input_dim, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Primeira camada convolucional com ReLU e max pooling
        x = self.pool1(F.relu(self.conv1(x)))

        # Segunda camada convolucional com ReLU e max pooling
        x = self.pool2(F.relu(self.conv2(x)))

        # Achatar a saída da camada convolucional para alimentar a camada totalmente conectada
        x = x.view(-1, self.fc_input_dim)

        # Camada totalmente conectada com ReLU
        x = F.relu(self.fc1(x))

        # Camada de saída
        x = self.fc2(x)

        return x

    def calculate_fc_input_dim(self):
        # Cria uma entrada de exemplo e passa pelas camadas convolucionais
        example_input = torch.zeros(1, 1, 28, 28)
        x = self.pool1(F.relu(self.conv1(example_input)))
        x = self.pool2(F.relu(self.conv2(x)))

        # Obtém a forma da saída das camadas convolucionais
        output_shape = x.shape
        fc_input_dim = output_shape[1] * output_shape[2] * output_shape[3]
        return fc_input_dim

  
 

model_CNN_2=MNISTModel_CNN_2()
print(model_CNN_2)


 

'''

# Treinamento o Modelo CNN


In [ ]:
criterion = nn.CrossEntropyLoss()       # Loss function 
parametes_model=model_CNN_1.parameters()

optimizer = torch.optim.Adam(parametes_model, lr=0.001)  # Optimizer
patience = 6  # Número de épocas para esperar antes de parar

train_model(model_CNN_1, criterion, optimizer, train_loader_CNN, val_loader_CNN, patience)


In [ ]:
# Inicialize o modelo 
model_CNN_1 = MNISTModel_CNN_1()

# Carregue o estado do modelo salvo
model_CNN_1.load_state_dict(torch.load('/kaggle/working/best_model_CNN.pth'))

model_CNN_1.eval()




# Avaliação do Modelo

##  Métricas de avaliação de modelos de classificação

Para calcular as metricas de avaliação usamos os sequintes valores:

**Verdadeiros Positivos (TP)**: Verdadeiros positivos são os casos em que o modelo previu a classe positiva corretamente. Em outras palavras, a classe real do exemplo era positiva e o modelo também previu a classe como positiva.

**Falsos Positivos (FP)**: Falsos positivos são os casos em que o modelo previu incorretamente a classe positiva. Isso significa que a classe real do exemplo era negativa, mas o modelo previu a classe como positiva. Isso é também conhecido como um erro do Tipo I.

**Verdadeiros Negativos (TN)**: Verdadeiros negativos são os casos em que o modelo previu a classe negativa corretamente. Em outras palavras, a classe real do exemplo era negativa e o modelo também previu a classe como negativa.

**Falsos Negativos (FN)**: Falsos negativos são os casos em que o modelo previu incorretamente a classe negativa. Isso significa que a classe real do exemplo era positiva, mas o modelo previu a classe como negativa. Isso é também conhecido como um erro do Tipo II.


Esses quatro valores formam a base da matriz de confusão. Além disso, várias métricas de avaliação, como precisão, recall e F1 score, são calculadas com base nesses valores. 

## Métricas


1. **Accuracy (Acurácia)**: É a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões. É uma métrica útil quando as classes estão bem balanceadas, mas pode ser enganosa quando as classes estão desbalanceadas. A fórmula para a acurácia é:

   $$ \text{Accuracy} = \frac{\text{Verdadeiros Positivos (TP) + Verdadeiros Negativos (TN)}}{\text{Verdadeiros Positivos (TP) + Falsos Positivos (FP) + Verdadeiros Negativos (TN) + Falsos Negativos (FN)}} $$
   


2. **Precision (Precisão)**: É a proporção de previsões positivas que são realmente corretas. É uma métrica importante quando o custo de Falsos Positivos é alto. A fórmula para a precisão é:

   $$ \text{Precision} = \frac{\text{Verdadeiros Positivos (TP)}}{\text{Verdadeiros Positivos (TP) + Falsos Positivos (FP)}} $$


3. **Recall (Revocação ou Sensibilidade)**: É a proporção de positivos reais que foram identificados corretamente. É uma métrica importante quando o custo de Falsos Negativos é alto. A fórmula para o recall é:

   $$ \text{Recall} = \frac{\text{Verdadeiros Positivos (TP)}}{\text{Verdadeiros Positivos (TP) + Falsos Negativos (FN)}} $$


4. **F1 Score**: É a média harmônica entre a precisão e o recall. É uma métrica útil quando você precisa de um equilíbrio entre a precisão e o recall e há uma distribuição desigual de classes. A fórmula para o F1 Score é:

   $$ \text{F1 Score} = 2 * \frac{\text{Precision * Recall}}{\text{Precision + Recall}} $$


Essas métricas são comumente usadas para avaliar modelos de classificação e cada uma tem seus próprios pontos fortes e fracos, dependendo da situação específica.


5. **Coeficiente de Kappa de Cohen**

O coeficiente de Kappa de Cohen é uma estatística que é usada para medir a precisão de classificação em tarefas de classificação. É mais útil quando os dados são classificados por humanos, pois leva em consideração a possibilidade de o acordo ocorrer por acaso.

O coeficiente de Kappa de Cohen varia de -1 a 1. Um valor de 1 indica que há um acordo perfeito entre os classificadores. Um valor de 0 indica que o acordo é o mesmo que seria esperado por acaso. Um valor negativo indica que o acordo é pior do que o aleatório.

Aqui está como o coeficiente de Kappa de Cohen é geralmente interpretado:

- Valores ≤ 0: Nenhum acordo
- 0.01–0.20: Nenhum a um leve acordo
- 0.21–0.40: Acordo justo
- 0.41–0.60: Acordo moderado
- 0.61–0.80: Acordo substancial
- 0.81–0.99: Acordo quase perfeito
- 1: Acordo perfeito

O coeficiente de Kappa de Cohen é uma medida mais robusta do que a simples porcentagem de concordância, porque leva em consideração a possibilidade de o acordo ocorrer por acaso. Isso é especialmente importante quando os dados estão desequilibrados.


6. **Coeficiente de Correlação de Matthews (MCC)**

O Coeficiente de Correlação de Matthews (MCC) é uma medida de qualidade para problemas de classificação binária. Ele leva em consideração verdadeiros e falsos positivos e negativos e é geralmente considerado uma medida equilibrada, o que significa que pode ser usado mesmo se as classes estiverem de tamanhos muito diferentes.

O MCC é, em essência, uma correlação entre as observações reais e as previsões: um coeficiente de +1 representa uma previsão perfeita, 0 uma previsão média aleatória e -1 uma previsão inversa.

A fórmula para o MCC é:

$$ MCC = \frac{(TP*TN - FP*FN) }{ \sqrt{(TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)} }$$



O denominador da fórmula garante que o MCC sempre caia entre -1 e +1.

7. **Matriz de Confusão**

A Matriz de Confusão é uma tabela usada para descrever o desempenho de um modelo de classificação em um conjunto de dados para os quais os valores verdadeiros são conhecidos. Ela é chamada de matriz de confusão porque permite visualizar facilmente o tipo de confusão que o classificador está causando, mostrando onde o classificador está confundindo uma classe por outra.

 

Para uma classificação binária a matriz de confusão é uma tabela 2x2. As linhas da matriz representam as classes reais, enquanto as colunas representam as classes previstas pelo modelo. A matriz é organizada da seguinte forma:


|                     | **Previsto: Positivo** | **Previsto: Negativo** |
|---------------------|------------------------|------------------------|
| **Real: Positivo**  | Verdadeiros Positivos  | Falsos Negativos       |
| **Real: Negativo**  | Falsos Positivos       | Verdadeiros Negativos  |


A matriz de confusão é uma ferramenta poderosa para entender como seu modelo está performando e onde ele está cometendo erros. Além disso, muitas métricas de avaliação do modelo, como precisão, recall e F1 score, são calculadas com base nos valores da matriz de confusão.

7. **ROC Curve**

A curva ROC (Receiver Operating Characteristic) é uma ferramenta gráfica usada para avaliar o desempenho de um modelo de classificação binária ou diagnóstico. Ela foi desenvolvida durante a Segunda Guerra Mundial para a análise de sinais de radar e desde então tem sido usada em muitos outros campos.

A curva ROC é um gráfico de duas dimensões onde a taxa de verdadeiros positivos (TPR, True Positive Rate) é plotada no eixo Y e a taxa de falsos positivos (FPR, False Positive Rate) é plotada no eixo X.  

Cada ponto na curva ROC representa um par de valores (FPR, TPR) correspondentes a um limiar de decisão específico. O limiar de decisão é o valor a partir do qual decidimos se uma previsão é classificada como positiva ou negativa. Ao variar o limiar de decisão, obtemos diferentes pares de valores (FPR, TPR) que formam a curva ROC.

A linha diagonal do canto inferior esquerdo ao canto superior direito representa a curva ROC de um classificador aleatório (um classificador que faz previsões aleatórias). Um bom classificador tem a curva ROC mais próxima do canto superior esquerdo (alta taxa de verdadeiros positivos e baixa taxa de falsos positivos).

A área sob a curva ROC (AUC, Area Under the Curve) é uma métrica que resume o desempenho do classificador. A AUC varia de 0 a 1, onde 1 representa um classificador perfeito e 0.5 representa um classificador aleatório. Quanto maior a AUC, melhor o classificador.





- **Binarização dos rótulos**: Os rótulos verdadeiros (neste caso os labels 0-9) são binarizados usando a função `label_binarize` do sklearn. Isso é necessário porque a curva ROC e a AUC são calculadas para a classificação binária e precisamos de uma representação binária para a classificação multiclasse.

- **Cálculo da curva ROC e AUC**: O código então entra em um loop sobre o número de classes. Para cada classe, ele calcula a curva ROC e a AUC usando as funções `roc_curve` e `auc` do sklearn, respectivamente.  
 
8. **PR Curve**

A Curva de Precisão-Recall (PR Curve) é uma ferramenta gráfica usada para avaliar o desempenho de um modelo de classificação em termos de precisão e recall. É especialmente útil em situações onde as classes estão muito desbalanceadas.

A Curva de Precisão-Recall é um gráfico de duas dimensões onde a precisão é plotada no eixo Y e o recall (também conhecido como taxa de verdadeiros positivos) é plotado no eixo X.
 
Cada ponto na curva PR representa um par de valores (Recall, Precisão) correspondentes a um limiar de decisão específico. O limiar de decisão é o valor a partir do qual decidimos se uma previsão é classificada como positiva ou negativa. Ao variar o limiar de decisão, obtemos diferentes pares de valores (Recall, Precisão) que formam a curva PR.

A área sob a curva PR (AUC-PR) é uma métrica que resume o desempenho do classificador. Quanto maior a AUC-PR, melhor o classificador. A AUC-PR é especialmente útil quando as classes estão desbalanceadas, pois leva em conta tanto a precisão quanto o recall, ao contrário da AUC-ROC, que pode ser excessivamente otimista em tais situações.
 
 
 A curva de ROC e a AUC-PR são calculadas para problemas de **classificação binária**. Se você está trabalhando com um problema de classificação multiclasse, você pode precisar adaptar o código para calcular a curva de Precisão-Recall e AUC-PR para cada classe individualmente.
  


In [ ]:


def calculate_roc_auc(model, dataloader):
    model.eval()
    y_test = []
    y_score = []

    # Iterar sobre os dados do dataloader
    for images, labels in dataloader:
        # Fazer a predição do modelo
        outputs = model(images)
        # Aplicar softmax para obter as probabilidades
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        # Adicionar aos arrays
        y_test.append(labels.numpy())
        y_score.append(probabilities.detach().numpy())

    y_test = np.concatenate(y_test)
    y_score = np.concatenate(y_score)

    # Binarizar as labels
    y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    # Calcular ROC para cada classe
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Calcular Precision-Recall para cada classe
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(10):
        precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_test_bin[:, i], y_score[:, i])

    # Plotar ROC e PR para todas as classes em um único gráfico
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for i in range(10):
        axs[0].plot(fpr[i], tpr[i], label='Class %d (area = %0.5f)' % (i, roc_auc[i]))
        axs[1].plot(recall[i], precision[i], label='Class %d (area = %0.5f)' % (i, average_precision[i]))
    
    ####################################################
    axs[0].plot([0, 1], [0, 1], 'k--')
    axs[0].set_xlim([0.0, 1.0])
    axs[0].set_ylim([0.0, 1.05])
    axs[0].set_xlabel('False Positive Rate')
    axs[0].set_ylabel('True Positive Rate')
    axs[0].set_title('Receiver Operating Characteristic')
    axs[0].legend(loc="lower right")

    axs[1].set_xlim([0.0, 1.0])
    axs[1].set_ylim([0.0, 1.05])
    axs[1].set_xlabel('Recall')
    axs[1].set_ylabel('Precision')
    axs[1].set_title('Precision-Recall curve')
    axs[1].legend(loc="lower right")
    
    ####################################################
    #ZOOM
    axs[0].set_xlim([0.0, 0.5])
    axs[0].set_ylim([0.9, 1.05])   
    
    axs[1].set_xlim([0.90, 1.05])
    axs[1].set_ylim([0.5, 1.05])
    ####################################################

    plt.tight_layout()
    plt.show()

    model.train()





In [ ]:

def evaluate_model(model, val_loader):
    model.eval()  # Set the model to evaluation mode
    true_labels = []
    predictions = []
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.numpy())
            predictions.extend(predicted.numpy())


    # Accuracy
    accuracy = accuracy_score(true_labels, predictions)
    print("Accuracy: ",  round(accuracy, 3))

    # Precision
    precision = precision_score(true_labels, predictions, average='weighted')
    print("Precision: ",  round(precision, 3))

    # Recall
    recall = recall_score(true_labels, predictions, average='weighted')
    print("Recall: ",  round(recall, 3))

    # F1 Score
    f1 = f1_score(true_labels, predictions, average='weighted')
    print("F1 Score: ",  round(f1, 3))
    
#############################################################
#    Está desativado pq não é um problema de classificação binaria (temos 10 classes)
#    # Matthews Correlation Coefficient (MCC)
#    mcc = matthews_corrcoef(true_labels, predictions)
#    print("Matthews Correlation Coefficient: ",  round(mcc, 3))
#############################################################

    # Cohen's Kappa
    kappa = cohen_kappa_score(true_labels, predictions)
    #interpret_kappa
    print("Cohen's Kappa: ",    round(kappa, 3))
    if kappa <= 0:
        print ("Nenhum acordo")
    elif kappa <= 0.20:
        print ("Nenhum a um leve acordo")
    elif kappa <= 0.40:
        print ("Acordo justo")
    elif kappa <= 0.60:
        print ("Acordo moderado")
    elif kappa <= 0.80:
        print ("Acordo substancial")
    elif kappa < 1:
        print ("Acordo quase perfeito")
    elif kappa == 1:
        print ("Acordo perfeito")
    else:
        print ("Valor de Kappa inválido")



    # Classification Report
    report = classification_report(true_labels, predictions)
    print("Classification Report:\n", report)

    # Confusion Matrix
    cm = confusion_matrix(true_labels, predictions)
    #print("Confusion Matrix:\n", cm)
    
    # Plotting Confusion Matrix
    plt.figure(figsize=(10,7))
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.show()
    
    # Curva de ROC e a AUC-PR
    calculate_roc_auc(model, val_loader)
 




In [ ]:
evaluate_model(model_CNN_1, val_loader_CNN)

## Prevendo e identificando erros

In [ ]:

def predict_and_identify_errors(model, val_loader):
    model.eval()  # Defina o modelo para o modo de avaliação
    correct = 0
    total = 0
    wrong_predictions = []

    with torch.no_grad():  # Sem calcular gradientes  
        for images, labels in val_loader:
            outputs = model(images)  # Obtenha as previsões do modelo
            _, predicted = torch.max(outputs.data, 1) 
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Identificar as previsões erradas
            wrong_indices = (predicted != labels).nonzero()[:, 0] 
            #.nonzero() retorna os índices dos elementos diferentes de zero. 
            wrong_images = images[wrong_indices]
            wrong_labels = predicted[wrong_indices]
            true_labels = labels[wrong_indices]
            wrong_predictions.extend(list(zip(wrong_images, wrong_labels, true_labels)))

 
    # Mostra as previsões erradas
    print("Total de previsões erradas:", total - correct )
    print("Previsões erradas:")
    
    # Para 2o imagens, dispostas em 2 linhas e 10 colunas
    n_rows = 3
    n_cols = 10
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols, n_rows+0.3))  # Create a subplot with 1 row and 20 columns
    axs_max=len(list(axs.flat))

    for i, (img, wrong_label, true_label) in enumerate(wrong_predictions[:axs_max]): 
        # Calcula a linha e a coluna atual
        row = i // n_cols
        col = i % n_cols
    
        # Plot das imagens erradas
        axs[row, col].imshow(img.view(28, 28), cmap='binary')
        axs[row, col].axis('off')   
        axs[row, col].set_title(f'P: {wrong_label.item()}, T: {true_label.item()}')  # Defina o título do subplot
    plt.show()  # Show the plot




In [ ]:
predict_and_identify_errors(model_CNN_1, val_loader_CNN)


# Submissão

In [ ]:

def submission_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    with torch.no_grad():
        for images in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.numpy())
           # print('oi')
        # Create a DataFrame with the predictions
        df = pd.DataFrame(predictions, columns=['Label'])
        df['Label'] = df['Label'].astype(int)  
        df.index.name = 'ImageId'
        df.index += 1  # Make the index start at 1 instead of 0
    # Save the DataFrame to a CSV file
    df.to_csv('submission.csv')
    return(df)

submission_model(model_CNN_1, test_loader_CNN)
 

 
# FIM

In [ ]:
#os.remove("/kaggle/working/submission.csv")